# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from IPython.display import display

data = pd.read_csv('/datasets/users_behavior.csv')
display(data)

data.info()
print(data.head())
print('')
print(data.tail())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

      calls  minutes  messages   mb_used  is_ultra
3209  122.0   910.98      20.0  35124.90         1
3210   25.0   190.36       0.0   3275.61         0
3211   97.0   634.44      70.0  13974.06         0
3212   64.0   462.32      90.0  31239.78         0
3213   80.0   566.09       6.0  29480.52         1


## 2. Разбейте данные на выборки

In [2]:
from sklearn.model_selection import train_test_split 

data_study, data_other = train_test_split(data, test_size=0.4, random_state=12345)
#ОБУЧАЮЩАЯ ВЫБОРКА
data_train_features = data_study.drop(['is_ultra'], axis = 1)
data_train_target = data_study['is_ultra']

#ВАЛИДАЦИОННАЯ ВЫБОРКА
data_valid, data_train = train_test_split(data_other, test_size = 0.5, random_state=12345)
data_valid_features = data_valid.drop(['is_ultra'], axis=1)
data_valid_target = data_valid['is_ultra']

#ТЕСТОВАЯ ВЫБОРКА
data_test_features = data_train.drop(['is_ultra'], axis=1)
data_test_target = data_train['is_ultra']

Было произведено разбиение исходного датасета на:
- <b>*обучающую выборку;*</b> 
- <b>*валидационную;*</b>
- <b>*тестовую*.</b>

## 3. Исследуйте модели

In [3]:
from sklearn.tree import DecisionTreeClassifier #дерево решений

model_1_0 = DecisionTreeClassifier(random_state = 12345)
model_1_0.fit(data_train_features, data_train_target)

valid_prediction = model_1_0.predict(data_valid_features)

print("Предсказания:", valid_prediction[0:11])
print("Правильные ответы:", data_valid_target.values[0:11])

def error_count(answer, predictions):
    error = 0
    for i in range(len(answer)):
        if answer[i] != predictions[i]:
            error += 1
    return error

print('Ошибок:', error_count(data_valid_target.values, valid_prediction))

def accuracy(answers, predictions):
    correct = 0
    for i in range(len(answers)):
        if answers[i] == predictions[i]:
            correct += 1
    return correct/len(answers)

print('Accuracy валидационная выборка:', accuracy(data_valid_target.values, valid_prediction))

train_prediction = model_1_0.predict(data_train_features)

print('Accuracy обучающая выборка:', accuracy(data_train_target.values, train_prediction))

Предсказания: [0 0 0 0 0 0 0 0 1 0 0]
Правильные ответы: [0 0 0 0 0 1 1 0 1 0 0]
Ошибок: 184
Accuracy валидационная выборка: 0.713841368584759
Accuracy обучающая выборка: 1.0


На обучающей модели модель прогнозирует лучше, чем на валидационной(1 против 0.7138).

In [4]:
from sklearn.metrics import accuracy_score
for depth in range(1, 26):
    model_1_1 = DecisionTreeClassifier(random_state = 12345, max_depth = depth)# < создайте модель, указав max_depth=depth >

    model_1_1.fit(data_train_features, data_train_target)# < обучите модель >

    predictions_valid = model_1_1.predict(data_valid_features)# < найдите предсказания на валидационной выборке >

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(data_valid_target, predictions_valid)) 

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633
max_depth = 11 : 0.7620528771384136
max_depth = 12 : 0.7620528771384136
max_depth = 13 : 0.7558320373250389
max_depth = 14 : 0.7589424572317263
max_depth = 15 : 0.7465007776049767
max_depth = 16 : 0.7340590979782271
max_depth = 17 : 0.7356143079315708
max_depth = 18 : 0.7309486780715396
max_depth = 19 : 0.7278382581648523
max_depth = 20 : 0.7216174183514774
max_depth = 21 : 0.7278382581648523
max_depth = 22 : 0.7262830482115086
max_depth = 23 : 0.7169517884914464
max_depth = 24 : 0.713841368584759
max_depth = 25 : 0.713841368584759


В данном случае, что нет смысла увеличивать глубину нашего дерева, так как самый хороший результат(по параметру accuracy) достигается при глубине равной 3.

In [5]:
from sklearn.metrics import accuracy_score

best_model_2 = None
best_result_2 = 0
b = 0
for depth in range(1, 51):
    model_2 = DecisionTreeClassifier(random_state = 12345, max_depth = depth)# < создайте модель, указав max_depth=depth >

    model_2.fit(data_train_features, data_train_target)# < обучите модель >

#    predictions_valid = model.predict(data_valid_features)# < найдите предсказания на валидационной выборке >

    result_2 = model_2.score(data_valid_features, data_valid_target) # посчитайте качество модели на валидационной выборке
    if result_2 > best_result_2:
        best_model_2 = model_2# сохраните наилучшую модель
        best_result_2 = result_2#  сохраните наилучшее значение метрики accuracy на валидационных данных
        b = depth
        print(best_result_2, depth)
    #print("max_depth =", depth, ": ", end='')
    #print(accuracy_score(data_valid_target, predictions_valid)) 
print("Accuracy наилучшей модели дерева решений на валидационной выборке:", best_result_2)
print("Глубина на которой достигается самая высокая точность:", b)

0.7542768273716952 1
0.7822706065318819 2
0.7853810264385692 3
Accuracy наилучшей модели дерева решений на валидационной выборке: 0.7853810264385692
Глубина на которой достигается самая высокая точность: 3


<div class="alert alert-info" style="border:solid blue 2px; padding: 20px"> <b>Комментарий студента после первого ревью:</b> модель с лучшей выборкой получается при значении параметра depth = 3.</div>

Перебор помог улучшить модель, путём изменения гиперпараметра. 

Следующим шагом будет: обучение модель случайного леса.

In [6]:
from sklearn.ensemble import RandomForestClassifier #случайный лес

best_model_3_0 = None
best_result_3_0 = 0
for est in range(1, 51):
    model_3_0 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_0.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
    result_3_0 = model_3_0.score(data_valid_features, data_valid_target) # посчитайте качество модели на валидационной выборке
    if result_3_0 > best_result_3_0:
        best_model_3_0 = model_3_0# сохраните наилучшую модель
        best_result_3_0 = result_3_0#  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_0)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7947122861586314


In [7]:
#from sklearn.ensemble import RandomForestClassifier #случайный лес

best_model_3_1 = None
best_result_3_1 = 0
for est in range(1, 101):
    model_3_1 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_1.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
    result_3_1 = model_3_1.score(data_valid_features, data_valid_target) # посчитайте качество модели на валидационной выборке
    if result_3_1 > best_result_3_1:
        best_model_3_1 = model_3_1 # сохраните наилучшую модель
        best_result_3_1 = result_3_1 #  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_1)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7947122861586314


In [8]:
#from sklearn.ensemble import RandomForestClassifier #случайный лес

best_model_3_2 = None
best_result_3_2 = 0
for est in range(1, 151):
    model_3_2 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_2.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
    result_3_2 = model_3_2.score(data_valid_features, data_valid_target) # посчитайте качество модели на валидационной выборке
    if result_3_2 > best_result_3_2:
        best_model_3_2 = model_3_2# сохраните наилучшую модель
        best_result_3_2 = result_3_2#  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_2)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7947122861586314


In [9]:
#from sklearn.ensemble import RandomForestClassifier #случайный лес

best_model_3_3 = None
best_result_3_3 = 0
for est in range(1, 201):
    model_3_3 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_3.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
    result_3_3 = model_3_3.score(data_valid_features, data_valid_target) # посчитайте качество модели на валидационной выборке
    if result_3_3 > best_result_3_3:
        best_model_3_3 = model_3_3 # сохраните наилучшую модель
        best_result_3_3 = result_3_3 #  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_3)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7947122861586314


<div class="alert alert-info" style="border:solid blue 2px; padding: 20px"> <b>Количество деревьев от 10 до 201 с шагом 10:</b></div>

In [10]:
from sklearn.ensemble import RandomForestClassifier #случайный лес

best_model_3_0_0 = None
best_result_3_0_0 = 0
for est in range(10, 201, 10):
    model_3_0_0 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_0_0.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
    result_3_0_0 = model_3_0.score(data_valid_features, data_valid_target) # посчитайте качество модели на валидационной выборке
    if result_3_0_0 > best_result_3_0:
        best_model_3_0_0 = model_3_0# сохраните наилучшую модель
        best_result_3_0_0 = result_3_0#  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_0)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7947122861586314


<div class="alert alert-info" style="border:solid blue 2px; padding: 20px"> <b>Комментарий студента после первого ревью:</b> использование количества деревьев от 10 до 201 с шагом 10 дало результат хуже, чем использование количества деревьев от 1 до 51 с шагом 1. В связи с этим думаю, что лучше оставить так, как было от 1 до 51 с шагом 1.</div>

В результате эксперимента с количеством деревьев, мы можем увидеть, что нет смысла брать слишком много деревьев, так как самый успешный результат машина выдаёт нам уже при количестве деревьев = 51(возможно и меньше).

Как и было в лекциях и в заданиях тренажёра, можем видеть, что модель случайного леса пока является самой "успешной".

Теперь попробуем логистическую регрессию.

In [11]:
from sklearn.linear_model import LogisticRegression #логистическая регрессия

model_4 = LogisticRegression(random_state=12345) # инициализируйте модель логистической регрессии с параметром random_state=12345
model_4.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
result_4 = model_4.score(data_valid_features, data_valid_target) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result_4)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


По результатам проведённых нами исследований самой успешной моделью является модель случайного леса, чего и следовало ожидать, но он же оказалось самый "долгой".

<div class="alert alert-info" style="border:solid blue 2px; padding: 20px"> <b>Комментарий студента после первого ревью:</b> по результатам анализа лучшей моделью по метрике accuracy оказался случайный лес. В 4м шаге тестировать будем модель случайного леса, обучая модель объединив train+valid, затем проверим на выборке test.</div>

## 4. Проверьте модель на тестовой выборке

In [12]:
'''
#ТЕСТОВАЯ ВЫБОРКА
data_test_features = data_train.drop(['is_ultra'], axis=1)
data_test_target = data_train['is_ultra']
'''

from sklearn.metrics import accuracy_score
for depth in range(1, 26):
    model_1_2 = DecisionTreeClassifier(random_state = 12345, max_depth = depth)# < создайте модель, указав max_depth=depth >

    model_1_2.fit(data_train_features, data_train_target)# < обучите модель >

    predictions_test = model_1_2.predict(data_test_features)# < найдите предсказания на валидационной выборке >

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(data_test_target, predictions_test)) 

max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255
max_depth = 6 : 0.776049766718507
max_depth = 7 : 0.7993779160186625
max_depth = 8 : 0.7931570762052877
max_depth = 9 : 0.7807153965785381
max_depth = 10 : 0.7884914463452566
max_depth = 11 : 0.7744945567651633
max_depth = 12 : 0.7807153965785381
max_depth = 13 : 0.7713841368584758
max_depth = 14 : 0.76049766718507
max_depth = 15 : 0.7713841368584758
max_depth = 16 : 0.749611197511664
max_depth = 17 : 0.7480559875583204
max_depth = 18 : 0.749611197511664
max_depth = 19 : 0.7433903576982893
max_depth = 20 : 0.7325038880248833
max_depth = 21 : 0.7387247278382582
max_depth = 22 : 0.7293934681181959
max_depth = 23 : 0.7293934681181959
max_depth = 24 : 0.7309486780715396
max_depth = 25 : 0.7309486780715396


In [13]:
from sklearn.metrics import accuracy_score

best_model_2_1 = None
best_result_2_1 = 0
for depth in range(1, 51):
    model_2_1 = DecisionTreeClassifier(random_state = 12345, max_depth = depth)# < создайте модель, указав max_depth=depth >

    model_2_1.fit(data_train_features, data_train_target)# < обучите модель >

#    predictions_valid = model.predict(data_valid_features)# < найдите предсказания на валидационной выборке >

    result_2_1 = model_2_1.score(data_test_features, data_test_target) # посчитайте качество модели на валидационной выборке
    if result_2_1 > best_result_2_1:
        best_model_2_1 = model_2_1# сохраните наилучшую модель
        best_result_2_1 = result_2_1#  сохраните наилучшее значение метрики accuracy на валидационных данных
        print(best_result_2_1)
    #print("max_depth =", depth, ": ", end='')
    #print(accuracy_score(data_valid_target, predictions_valid)) 
print("Accuracy наилучшей модели дерева решений на валидационной выборке:", best_result_2_1)

0.7356143079315708
0.7744945567651633
0.7791601866251944
0.7838258164852255
0.7993779160186625
Accuracy наилучшей модели дерева решений на валидационной выборке: 0.7993779160186625


В данном случае можно увидеть, что на тестовой выборке максимально удачный результат получается при глубине дерева до 25(включительно), как и на валидационной выборке.

In [14]:
from sklearn.ensemble import RandomForestClassifier #случайный лес

best_model_3_0_1 = None
best_result_3_0_1 = 0
for est in range(1, 51):
    model_3_0_1 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_0_1.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
    result_3_0_1 = model_3_0_1.score(data_test_features, data_test_target) # посчитайте качество модели на валидационной выборке
    if result_3_0_1 > best_result_3_0_1:
        best_model_3_0_1 = model_3_0_1# сохраните наилучшую модель
        best_result_3_0_1 = result_3_0_1#  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_0_1)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7962674961119751


<div class="alert alert-info" style="border:solid blue 2px; padding: 20px"> <b>Комментарий студента после первого ревью:</b> проверка модели.</div>

In [15]:
from sklearn.ensemble import RandomForestClassifier #случайный лес



data_study_0, data_other_0 = train_test_split(data, test_size=0.2, random_state=12345)
#ОБУЧАЮЩАЯ ВЫБОРКА
data_train_features_0 = data_study_0.drop(['is_ultra'], axis = 1)
data_train_target_0 = data_study_0['is_ultra']
'''
#ВАЛИДАЦИОННАЯ ВЫБОРКА
data_valid, data_train = train_test_split(data_other, test_size = 0.5, random_state=12345)
data_valid_features = data_valid.drop(['is_ultra'], axis=1)
data_valid_target = data_valid['is_ultra']
'''

#ТЕСТОВАЯ ВЫБОРКА
data_test_features_0 = data_other_0.drop(['is_ultra'], axis=1)
data_test_target_0 = data_other_0['is_ultra']



best_model_3_0_1_1 = None
best_result_3_0_1_1 = 0
for est in range(1, 51):
    model_3_0_1_1 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_0_1_1.fit(data_train_features_0, data_train_target_0) # обучите модель на тренировочной выборке
    result_3_0_1_1 = model_3_0_1_1.score(data_test_features_0, data_test_target_0) # посчитайте качество модели на валидационной выборке
    if result_3_0_1_1 > best_result_3_0_1_1:
        best_model_3_0_1_1 = model_3_0_1_1# сохраните наилучшую модель
        best_result_3_0_1_1 = result_3_0_1_1#  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_0_1_1)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7884914463452566


<div class="alert alert-info" style="border:solid blue 2px; padding: 20px"> <b>Комментарий студента после первого ревью:</b> окончание проверки. Ниже приведени выводы и вопросы.</div>

In [16]:
best_model_3_0_2 = None
best_result_3_0_2 = 0
for est in range(1, 101):
    model_3_0_2 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_0_2.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
    result_3_0_2 = model_3_0_2.score(data_test_features, data_test_target) # посчитайте качество модели на валидационной выборке
    if result_3_0_2 > best_result_3_0_2:
        best_model_3_0_2 = model_3_0_2# сохраните наилучшую модель
        best_result_3_0_2 = result_3_0_2#  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_0_2)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7962674961119751


In [17]:
best_model_3_0_3 = None
best_result_3_0_3 = 0
for est in range(1, 151):
    model_3_0_3 = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model_3_0_3.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
    result_3_0_3 = model_3_0_3.score(data_test_features, data_test_target) # посчитайте качество модели на валидационной выборке
    if result_3_0_3 > best_result_3_0_3:
        best_model_3_0_3 = model_3_0_3# сохраните наилучшую модель
        best_result_3_0_3 = result_3_0_3#  сохраните наилучшее значение метрики accuracy на валидационных данных
#        print(best_result)

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_3_0_3)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.7962674961119751


Как и на валидационной выборке результат на тестовой выборке так же хорош(не хуже), и количество деревьев по прежнему достаточно (50).

In [18]:
from sklearn.linear_model import LogisticRegression #логистическая регрессия

model_4_1 = LogisticRegression(random_state=12345) # инициализируйте модель логистической регрессии с параметром random_state=12345
model_4_1.fit(data_train_features, data_train_target) # обучите модель на тренировочной выборке
result_4_1 = model_4_1.score(data_test_features, data_test_target) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result_4_1)

Accuracy модели логистической регрессии на валидационной выборке: 0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Логистическая регрессия нас подвела, во-первых: результат хуже, чем на валидационной выборке, ну и во-вторых он ниже 0.75.

В целом, всё осталось по прежнему, лучше всего себя показывает модель случайного леса, но при всей её "удачности" она же и самая медленная.

In [22]:
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')
def train_model(**kwargs):
    model = RandomForestClassifier(random_state=12345, **kwargs) # обучите модель с заданным количеством деревьев
    model.fit(data_train_features,data_train_target) # обучите модель на тренировочной выборке
    predicted_valid = model.predict(data_valid_features)
    result= accuracy_score(data_valid_target,predicted_valid)
    if result > best_result['score']:
        best_result['score'] = result
        best_result['params'] = kwargs

In [31]:
%%time
best_result = {'score': 0, 'params': {}}
for depth in tqdm(range(2, 20, 4), desc='depth'):
    for est in tqdm(range(10, 100, 10), desc='est', leave=False):
        for mf in tqdm([None, 'log2', 'sqrt'], desc='max_features', leave=False):
            train_model(max_depth=depth, n_estimators=est, max_features=mf)


CPU times: user 22min 35s, sys: 13.2 s, total: 22min 49s
Wall time: 24min 53s


In [32]:
best_result

{'score': 0.8087091757387247,
 'params': {'max_depth': 8, 'n_estimators': 40, 'max_features': 'log2'}}

In [33]:
m = RandomForestClassifier(random_state=12345, **best_result['params'])
m.fit(data_train_features_0, data_train_target_0)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [34]:
accuracy_score(data_test_target_0, m.predict(data_test_features_0))

0.8009331259720062

Результат получен за 38 сек. Если перебирать параметры очень точно, отличие получим только в глубине (стало 8 против 6, время перебора 24 мин), увеличение accuracy на 0.01 - то есть профит небольшой (такое увеличение можно оставить на финальный тюнинг), а затраты по времени огромны

<hr style="border: 2px solid green;" />

In [19]:
data['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64